# DNBLab Jupyter Notebook Tutorial

## OAI2 - Schnittstellenabfrage, Datenauslieferung und Ergebnisanzeige

Dieses DNBLab-Tutorial beschreibt Beispielabfragen über die OAI2-Schnittstelle zu frei verfügbaren digitalen Objektsammlungen mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Anfrage und Ausgabe der Daten in den verschiedenen Metadatenformaten zur weiteren Verarbeitung. Die Daten werden über die OAI2-Schnittstelle standardmäßig als XML-Antwort in dem Format METS/MODS ausgeliefert. Zusätzlich steht die Auslieferung der Daten in Dublin Core (dc) zur Verfügung.

**Inhaltsverzeichnis:** 

* [1. Einrichten der Arbeitsumgebung](#Teil1)
* [2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum](#Teil2)
* [3. Zusätzliche Ausgabe der Links zu den Objekten](#Teil3)
* [4. Ausgabe der gescannten Bilder](#Teil4)

###1. Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die OAI2 Schnittstelle wird sickle https://sickle.readthedocs.io/en/latest/tutorial.html und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet.

In [ ]:
from sickle import Sickle
from lxml import etree

Die OAI2 Basis URL ist http://services.dnb.de/oai2/repository. 

In [2]:
sickle = Sickle('http://services.dnb.de/oai2/repository')

Im nächsten Schritt werden alle verfügbaren Objektsammlungen (Projekte) mit der Funktion ListSets abgefragt und angezeigt.

In [ ]:
oai_sets = sickle.ListSets()
for oai_set in oai_sets:
    print('setSpec value for selective harvesting: ' + oai_set.setSpec)
    print('Name of the set (setName): ' + oai_set.setName + '\n')

Die bisher zur Verfügung stehenden freien Objektsammlungen entsprechen folgenden Projekten (oai_sets):
 
| oai_set | Beschreibung | Anzeige im Katalog |
| --- | --- | --- |
| dnb:digitalisate-oa:projekt4 | 100.000 Seiten aus 30 Titeln, die ein breites Spektrum der Exilpresse 1933-1945 repräsentieren | <a href=https://portal.dnb.de/opac.htm?method=showPreviousResultSite&currentResultId=%22exilpresse%22+and+%22digital%22%26any%26dnb.dea.exp%26leipzig&currentPosition=10>Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt7 | Gemeinfreie Digitalisate der Exilsammlungen | <a href=https://portal.dnb.de/opac.htm?query=cod%3Dd007+location%3Donlinefree+&method=simpleSearch&cqlMode=true>Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt8 | 1Mehr als 1.200 Bücher, Broschüren, Plakate und andere Einblattdrucke zum Weltkrieg 1914-1918 | <a href=https://portal.dnb.de/opac.htm?query=cod%3Dd008+location%3Donlinefree+&method=simpleSearch&cqlMode=true>Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt26 | Mehr als 600 digitalisierte Quellen zur deutschen Revolution 1918/1919 und zur Nachkriegszeit | <a href=https://portal.dnb.de/opac.htm?query=cod%3Dd026+location%3Donlinefree+&method=simpleSearch&cqlMode=true>Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt28 | Digitalisierte Werke aus Architektur, Design, Malerei und Druckkunst von 1918 bis 1933 | <a href=https://portal.dnb.de/opac.htm?query=cod%3Dd028+location%3Donlinefree+&method=simpleSearch&cqlMode=true>Sammlung im Katalog</a>
| dnb:digitalisate-oa:projekt29 | Mehr als 2.400 Bildnisse von Buchhändlern, Buchdruckern und Verlegern des 17. bis 20. Jahrhunderts | <a href=https://portal.dnb.de/opac.htm?query=cod%3Dd029+location%3Donlinefree+&method=simpleSearch&cqlMode=true>Sammlung im Katalog</a>

Die verfügbaren Metadatenformate können mit der Funktion ListMetadataFormats abgefragt und mit print() ausgegeben werden. 


In [ ]:
oai_formats = sickle.ListMetadataFormats()
for oai_format in oai_formats:
    print(oai_format.metadataPrefix)

###2. Ausgabe von Titeln eines Projekts in bestimmten Zeitraum <a class="anchor" id="Teil2"></a>

Um beispielhaft alle Datensätze der Buchhändlerportraits (Projekt29) ab dem 01.04.2019 aufzulisten, müssen für die Abfrage Metadatenformat, Zeitraum und Projekt angegeben werden.
Die OAI Funktionen werden unter http://www.openarchives.org/OAI/2.0/ spezifiziert und auf der DNBLab Seite unter Funktionen beschrieben.
Die Daten befinden sich in der XML-Antwort als Inhalt zwischen beschreibenden <Tags>. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat dc, können bei Bedarf die dc-Spezifikationen http://www.openarchives.org/OAI/2.0/oai_dc/ und http://purl.org/dc/elements/1.1/ herangezogen werden.


In [30]:
namespaces = {
    'oai': 'http://www.openarchives.org/OAI/2.0/',
    'oai_dc': 'http://www.openarchives.org/OAI/2.0/oai_dc/',
    'dc': 'http://purl.org/dc/elements/1.1/'
}

count=0

for record in sickle.ListRecords(**{'metadataPrefix': 'oai_dc', 'from': '2019-04-30', 'set': 'dnb:digitalisate-oa:projekt29'}):
    
    if ('ger' in record.raw):
        #print('-' + record.raw + '-')
        tree = etree.ElementTree(record.xml)
        result = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
        if (result):
            count += 1
            author = tree.xpath('/oai:record/oai:metadata/oai_dc:dc/dc:title/text()', namespaces=namespaces)
            print(str(count) + ": " + result[0])
            if author:
                print(author[0])

1: Porträt Wilhelm Volkmann (1837 - 1896) : Heliogravüre / Bez. u.l.: Phot. Aufnahme von Georg Brokesch, Leipzig; u.r.: Photogravure R. Paulussen, Wien
Porträt Wilhelm Volkmann (1837 - 1896) : Heliogravüre / Bez. u.l.: Phot. Aufnahme von Georg Brokesch, Leipzig; u.r.: Photogravure R. Paulussen, Wien
2: Porträt Johann Baptist Strobel (1748 - 1805) und seine Kinder : Heliogravüre / Bez. u.r.: F. BRUCKMANN A. G. MÜNCHEN
Porträt Johann Baptist Strobel (1748 - 1805) und seine Kinder : Heliogravüre / Bez. u.r.: F. BRUCKMANN A. G. MÜNCHEN
3: Porträt Ernst A. Seemann (1829 - 1904) : Heliogravüre / Bez. u.M.: Photogravure R. Paulussen, Wien
Porträt Ernst A. Seemann (1829 - 1904) : Heliogravüre / Bez. u.M.: Photogravure R. Paulussen, Wien
4: Porträt Johann Leonhard Schrag (1783 - 1858) : Heliogravüre / Bez. u.l.: M.? Wolfsheimer pinx.; u.r.: Photogravure Bruckmann
Porträt Johann Leonhard Schrag (1783 - 1858) : Heliogravüre / Bez. u.l.: M.? Wolfsheimer pinx.; u.r.: Photogravure Bruckmann
5: Portr

###3. Zusätzliche Ausgabe der Links zu den Objekten <a class="anchor" id="Teil3"></a>

Damit zusätzlich zu den Titeln die Links zu den Objekten ausgegeben werden, wird der URN entsprechend des dc-Schemas /oai:record/oai:metadata/oai_dc:dc/dc:identifier/text() angesprochen und zusammen mit dem Titel durchnummeriert angezeigt. 

###4. Ausgabe der gescannten Bilder <a class="anchor" id="Teil4"></a>

Das Format mets bildet neben den bibliographischen Daten im Format mods die Struktur der digitalen Objekte bzw. die Reihenfolge der gescannten Bilder ab. Für die gezielte Abfrage der gewünschten Informationen aus dem Metadatenformat mets können bei Bedarf die mets-Spezifikationen http://www.loc.gov/METS/ und https://www.loc.gov/standards/mets/mets.xsd herangezogen werden. Die Links der einzelnen Bilder können über <xsd:element name="fileGrp"> angesprochen und zur weiteren Verarbeitung ausgegeben werden.